In [3]:
import math
import numpy as np
import pandas as pd
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA
import pmdarima as pm
import time
import xgboost as xgb

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from tqdm.notebook import tqdm

/Users/radu/.virtualenvs/ml/lib/python3.8/site-packages/statsforecast/core.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
full_df = pd.read_csv('data/m4/daily-train.csv')

def get_ts(full_df, index):
    df = full_df.iloc[index:index+1, 1:].transpose()
    df.columns = ['y']
    df = df[df['y'].notna()]
    return df

# df = get_ts(full_df, 6)
df = get_ts(full_df, 1)
df.tail()

,y
V1003,2978.0
V1004,2991.9
V1005,2995.3
V1006,3000.5
V1007,2968.5


In [5]:
def get_features_from_lags(df, feature_count=7):
    res = df[['y']].copy()
    lags = ['lag_{}'.format(lag) for lag in range(1, feature_count + 1)]
    
    # The lags are computed for the *previous* value because when we forecast,
    # we cannot compute them for the current value
    res['previous'] = res['y'].shift(1)
    for i, lag in enumerate(lags):
        res[lag] = res['previous'] - res['previous'].shift(i + 1)
    
    res['day'] = np.arange(res.shape[0]) % 7
    res['lag_to_predict'] = res['y'] - res['previous']
    
    # Ignore the first row, as it has no previous values, it cannot be predicted
    return res[1:].drop(['previous', 'y'], axis=1)

def get_future_lags(df, feature_count=7):
    res = df[['y']].copy()
    lags = ['lag_{}'.format(lag) for lag in range(1, feature_count + 1)]
    
    for i, lag in enumerate(lags):
        res[lag] = res['y'] - res['y'].shift(i + 1)
    
    res['day'] = np.arange(res.shape[0]) % 7
    
    return res

In [6]:
def train_xgb(df, in_sample_validation=False):
    xgb_reg_params = {
    #         'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.1)),
        'learning_rate': 0.1,
        'max_depth':        5,
        'early_stopping_rounds': 100,
        'eval_metric': 'rmse',
        'n_estimators': 100,
    }

    if in_sample_validation:
        # Validate against in sample data
        train = df
        validation = df
    else:
        # Validate against out of sample
        train = df[:-50]
        validation = df[-50:]

    X_train = train.drop('lag_to_predict', axis=1)
#     print(X_train)
    y_train = train[['lag_to_predict']]


    reg = xgb.XGBRegressor(**xgb_reg_params)
    eval_set = [(validation.drop('lag_to_predict', axis=1), validation[['lag_to_predict']])]
    reg.fit(X_train, y_train, eval_set=eval_set, verbose=False)
    
    return reg

def predict_xgb(df, model):
    return model.predict(df)


In [7]:
def predict_with_xgb(df, feature_func):
    lags = feature_func(df)
    xgb_model = train_xgb(lags)

    lags_for_predictions = get_future_lags(df)
    lag_prediction = predict_xgb(lags_for_predictions.iloc[-1:].drop(['y'], axis=1), xgb_model)
    tree_prediction = lags_for_predictions.iloc[-1:]['y'][0] + lag_prediction[0]
    
    return tree_prediction

In [8]:
def predict_all(df, feature_func, model = None):
    df = df.copy()
    
    # Step 1. Get the ARIMA forecast. 
    # For reasons of speed (when testing multiple models), it can be passed as a parameter so it's not
    # computed every time because it can take 1-2mins to compute for certain time series
    if model is None:
        model = pm.auto_arima(df['y'], seasonal=True, m=7)

    # Step 2. Compute the features to be used for residuals forecasting
    lags = feature_func(df)
    
    # Step 3. Predict the next value
    xgb_model = train_xgb(lags)
    
    
    """Steps to predict
        1) Find the lags of current row
        2) Find the predicted lag
        3) Add the lag to the actual value to get the final forecast
    """
    
    lags_for_predictions = get_future_lags(df)
    lag_prediction = predict_xgb(lags_for_predictions.iloc[-1:].drop(['y'], axis=1), xgb_model)
    tree_prediction = lags_for_predictions.iloc[-1:]['y'][0] + lag_prediction[0]
    
    
    # Step 4. Get the ARIMA 1 step ahead prediction
    arima_prediction = model.predict(1)
    
    # Step 5. Do an average between the two predictions
    return (arima_prediction[0] + tree_prediction) / 2

predict_all(df, get_features_from_lags)

2968.734728202224

In [6]:
def test_xgb_predictions(df):
    train_df = df[:-10]
    test_df = df[-10:].copy()

    model = train_xgb(train_df)
    test_df['yhat'] = predict_xgb(test_df.drop('y', axis=1), model)
    print("MSE:", mean_squared_error(test_df['y'], test_df['yhat']))
    return test_df

In [9]:
%%time
test_predictions = []

total_batches = 10
batch = 0

rows_per_batch = math.ceil(full_df.shape[0] / total_batches)
batch_start = rows_per_batch * batch
batch_end = min(batch_start + rows_per_batch, full_df.shape[0])

for i in tqdm(range(50)):
# for i in tqdm(range(batch_start, batch_end)):
    df1 = get_ts(full_df, i)
    # We'll try to predict the last value in the time series
    source_df = df1[:-1]
    
    start_time = time.time()
    model = pm.auto_arima(source_df['y'], seasonal=True, m=7, error_action='ignore')
    arima_prediction = model.predict(1)[0]
    
    arima_time = time.time()
    combined_prediction = predict_all(source_df, get_features_from_lags, model)
    xgb_prediction = predict_with_xgb(source_df, get_features_from_lags)
    
    end_time = time.time()
    
    result = {
        'y': df1[-1:]['y'][0],
        'yhat': arima_prediction,
        'yhat_xgb': xgb_prediction,
        'yhat_lags': combined_prediction,
        'arima_time': arima_time - start_time,
        'xgboost_time': end_time - arima_time,
        'total_time': end_time - start_time
    }
    
    result['error'] = result['y'] - result['yhat']
    result['error_lags'] = result['y'] - result['yhat_lags']
    result['error_xgb'] = result['y'] - result['yhat_xgb']

    test_predictions.append(result)

errors = pd.DataFrame(test_predictions)
# errors.to_csv('results_{}.csv'.format(batch))
errors

  0%|          | 0/50 [00:00<?, ?it/s]

CPU times: user 11min 32s, sys: 9min 30s, total: 21min 3s
Wall time: 10min 1s


,y,yhat,yhat_xgb,yhat_lags,arima_time,xgboost_time,total_time,error,error_lags,error_xgb
0,2029.70000,2032.610458,2032.167019,2032.388739,2.750032,1.590839,4.340871,-2.910458,-2.688739,-2.467019
1,2968.50000,3000.500000,3000.964791,3000.732396,2.522878,3.162980,5.685858,-32.000000,-32.232396,-32.464791
2,1123.50000,1126.300000,1123.301267,1124.800634,0.883128,2.508123,3.391251,-2.800000,-1.300634,0.198733
3,1218.00000,1208.960718,1244.362500,1226.661609,1.728160,1.087089,2.815249,9.039282,-8.661609,-26.362500
4,5868.55000,5884.274156,5875.325325,5879.799740,0.530730,1.447004,1.977734,-15.724156,-11.249740,-6.775325
5,3103.28140,3097.759418,3149.929665,3123.844541,14.528290,3.118109,17.646399,5.521982,-20.563141,-46.648265
6,5249.27900,5099.845013,5220.545223,5160.195118,48.115473,1.258581,49.374054,149.433987,89.083882,28.733777
7,1732.04000,1730.933610,1731.737920,1731.335765,5.393956,1.474485,6.868441,1.106390,0.704235,0.302080
8,1584.77000,1584.136483,1581.563121,1582.849802,5.461706,1.939944,7.401650,0.633517,1.920198,3.206879
9,2620.10000,2622.500000,2621.245610,2621.872805,1.414426,2.857000,4.271426,-2.400000,-1.772805,-1.145610


In [10]:
mean_squared_error(errors['y'], errors['yhat'])

1970.8453096242902

In [11]:
mean_squared_error(errors['y'], errors['yhat_lags'])

1573.6271806423063

In [12]:
mean_squared_error(errors['y'], errors['yhat_xgb'])

1450.3621922834163